<a href="https://colab.research.google.com/github/mashyko/NLP_Mecab/blob/master/mecab_janome_BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#日本語文書の単語トークン化：語彙の辞書とBOW（Bag-Of-Words）のOne-hot 表現

mecab-python を用いた例

In [1]:
!pip install mecab-python3

     |████████████████████████████████| 17.1MB 230kB/s 


mecab を用いて形態素解析をします

In [2]:
import MeCab
sentence = '私はワインを飲みますが、彼女はワインを飲みません。'

#分かち書き
tagger = MeCab.Tagger("-Owakati")

text = tagger.parse(sentence)
print('text = ',text)

text = text.replace('。','\n')
text = text.replace('、','\n') 

words =text.split(' ')
print (words)



text =  私 は ワイン を 飲み ます が 、 彼女 は ワイン を 飲み ませ ん 。 

['私', 'は', 'ワイン', 'を', '飲み', 'ます', 'が', '\n', '彼女', 'は', 'ワイン', 'を', '飲み', 'ませ', 'ん', '\n', '\n']


トークン（単語）を数値化します

In [3]:
import numpy as np

word_to_id = {}
id_to_word = {}
for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

corpus = np.array([word_to_id[w] for w in words])

print ('id_to_word \n', id_to_word)
print ('word_to_id \n', word_to_id)
print ('corpus \n',corpus)

id_to_word 
 {0: '私', 1: 'は', 2: 'ワイン', 3: 'を', 4: '飲み', 5: 'ます', 6: 'が', 7: '\n', 8: '彼女', 9: 'ませ', 10: 'ん'}
word_to_id 
 {'私': 0, 'は': 1, 'ワイン': 2, 'を': 3, '飲み': 4, 'ます': 5, 'が': 6, '\n': 7, '彼女': 8, 'ませ': 9, 'ん': 10}
corpus 
 [ 0  1  2  3  4  5  6  7  8  1  2  3  4  9 10  7  7]


one-hot 表現を作成します。

In [4]:
vocab_size = len(word_to_id)
N = corpus.shape[0]

one_hot = np.zeros((N, vocab_size), dtype=np.int32)
for idx, word_id in enumerate(corpus):
    one_hot[idx, word_id] = 1


print ('one-hot 表現 \n',one_hot)


def create_co_matrix(corpus, vocab_size, window_size=1):
    '''共起行列の作成

    :param corpus: コーパス（単語IDのリスト）
    :param vocab_size:語彙数
    :param window_size:ウィンドウサイズ（ウィンドウサイズが1のときは、単語の左右1単語がコンテキスト）
    :return: 共起行列
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

matrix = create_co_matrix(corpus, vocab_size, window_size=1)
print ('co-matrix = \n',matrix)

one-hot 表現 
 [[1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0]]
co-matrix = 
 [[0 1 0 0 0 0 0 0 0 0 0]
 [1 0 2 0 0 0 0 0 1 0 0]
 [0 2 0 2 0 0 0 0 0 0 0]
 [0 0 2 0 2 0 0 0 0 0 0]
 [0 0 0 2 0 1 0 0 0 1 0]
 [0 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 1 2 1 0 1]
 [0 1 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 1 0]]


Janomeによる形態素解析:
頻出単語の検出


In [5]:
!pip install janome

     |████████████████████████████████| 21.5MB 1.3MB/s 


文書データを私のgithub から　git clone します。

In [6]:
! git clone https://github.com/mashyko/NLP_Mecab
%cd NLP_Mecab
!ls 

Cloning into 'NLP_Mecab'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 152 (delta 23), reused 0 (delta 0), pack-reused 102
Receiving objects: 100% (152/152), 12.60 MiB | 20.17 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/content/NLP_Mecab
corpora		      gensim_tutor_English.ipynb  Mecab_janome.ipynb
datasets	      livedoor-news-data	  Mecab_Neologd_livedoor.ipynb
fasttext_intro.ipynb  Livedoor-text		  mecab_python.ipynb
Gensim_doc2vec.ipynb  MeCab_1.ipynb		  README.md
gensim_topic.ipynb    Mecab_Intro.ipynb


janome.tokenizer を用いて、文書 blog.txt をトークン化します。
トークンのリストと頻出度を表示



In [7]:
from janome.tokenizer import Tokenizer

blog = open("./corpora/blog.txt", "r")
txt = blog.read()

t = Tokenizer()

word_dic = {}
lines = txt.split("\r\n")
for line in lines:
    blog_txt = t.tokenize(line)
    for w in blog_txt:
        word = w.surface
        ps = w.part_of_speech
        if ps.find('名詞') < 0: continue
        if not word in word_dic:
            word_dic[word] = 0
        word_dic[word] += 1

keys = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
for word,cnt in keys[:50]:
    print("{0}({1}) ".format(word,cnt), end="")



AI(29) こと(20) セミナー(16) よう(13) 理解(13) 参加(10) の(10) 的(8) 説明(7) 大変(6) 仕組み(6) 内容(6) イメージ(5) 自分(5) プログラミング(5) 知識(5) 作成(5) サービス(4) 非常(4) 今回(4) もの(4) 人工(4) 知能(4) 学習(4) 技術(3) ため(3) 満足(3) 今後(3) 活用(3) 提供(3) 丁寧(3) 入門(3) 操作(3) 具体(3) 機械(3) 簡単(3) 体験(3) お話(2) 午後(2) 数学(2) これ(2) 私(2) 興味(2) 開発(2) 講師(2) ご(2) 講義(2) 事(2) 性(2) 感謝(2) 

janome.tokenizer を用いて、夏目漱石の小説「こころ」をトークン化します。

In [8]:
from janome.tokenizer import Tokenizer

blog = open("./corpora/kokoro.txt", "r")
txt = blog.read()

t = Tokenizer()

word_dic = {}
lines = txt.split("\r\n")
for line in lines:
    blog_txt = t.tokenize(line)
    for w in blog_txt:
        word = w.surface
        ps = w.part_of_speech
        if ps.find('名詞') < 0: continue
        if not word in word_dic:
            word_dic[word] = 0
        word_dic[word] += 1

keys = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
for word,cnt in keys[:50]:
    print("{0}({1}) ".format(word,cnt), end="")



私(2697) の(1483) 先生(597) 事(576) よう(523) それ(408) もの(393) 人(389) 奥さん(388) 時(379) 彼(314) 父(272) 自分(264) 二(263) 中(257) 何(251) 一(246) ん(241) うち(238) い(234) 十(201) 方(200) あなた(187) 母(171) 前(168) お嬢さん(166) 上(154) 気(150) 今(150) 顔(135) め(133) 言葉(128) ため(126) 三(123) 日(123) 眼(123) 字(122) そこ(120) 心(116) ［＃(113) ［＃「(112) 下げ(111) 見出し(111) ５(109) 妻(108) 口(107) 通り(105) お(105) 家(96) 間(93) 

janome.tokenizer を用いて、livedoor news の topic-news をトークン化します。

In [9]:
from janome.tokenizer import Tokenizer

blog = open("./livedoor-news-data/topic-news.txt", "r")
txt = blog.read()

t = Tokenizer()

word_dic = {}
lines = txt.split("\r\n")
for line in lines:
    blog_txt = t.tokenize(line)
    for w in blog_txt:
        word = w.surface
        ps = w.part_of_speech
        if ps.find('名詞') < 0: continue
        if not word in word_dic:
            word_dic[word] = 0
        word_dic[word] += 1

keys = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
for word,cnt in keys[:50]:
    print("{0}({1}) ".format(word,cnt), end="")



/(3711) .(2061) -(1848) ネット(1595) :(1578) の(1450) こと(1322) 日(1229) livedoor(1143) 関連(1124) 韓国(1069) 掲示板(1055) 記事(1050) ://(946) http(927) com(908) 人(894) news(874) detail(861) 声(845) 00(833) article(832) +(774) T(771) 0900(770) 日本(739) 情報(729) ん(603) 2012(600) 者(598) 氏(598) 批判(579) 話題(572) )(526) 番組(524) ユーザー(514) よう(510) AKB(455) 放送(451) ((440) 2(436) 月(435) 的(435) 年(434) これ(426) 発言(417) ニュース(417) 上(381) 12(379) 10(355) 

以上
